[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Moorea-AI/AI4ENG_UDEA/blob/main/01_Exploración_de_datos.ipynb)

EXPLORACIÓN DE DATOS PARA EL DATASET "Fraude en Transacciones Electrónicas" TOMADO DE: https://www.kaggle.com/competitions/ieee-fraud-detection/data?select=test_transaction

Presentado por: </br>
Aura Luz Moreno Díaz, Estudiante de Ingeniería Industrial </br>
Evelyn Zharick Saez Gallego, Estudiante de Ingeniería Ambiental</br>
Para la materia:<br>
INTRODUCCIÓN A LA INTELIGENCIA ARTIFICIAL PARA LAS CIENCIAS Y LA INGENIERIA<BR>
Con el profesor: RAÚL RAMOS POLLAN

In [12]:
#PASO 1: Importación de librerias que serán necesarias para la exploración de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import requests
import json

#Importacion de librerias para manipulación de datos para Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold,GroupKFold
import lightgbm as lgb
import gc

import warnings
warnings.filterwarnings('ignore')


In [13]:
# Cargando credenciales de Kaggel
{"username":"mooreamed","key":"e97f8db473e00551f7aea527af024391"}
with open('kaggle.json', 'w') as file:
    json.dump(data, file, indent=4)

!cp kaggle.json ~/.kaggle/

In [14]:
# Se descarga el dataset
!kaggle datasets download -d niangmohamed/ieeecis-fraud-detection

# Se descomprime el dataset
!unzip ieeecis-fraud-detection.zip

 96% 113M/118M [00:01<00:00, 83.6MB/s] 
100% 118M/118M [00:01<00:00, 75.6MB/s]
Archive:  ieeecis-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [15]:
#Verificamos que se este funcionando la conexión de la API
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
salvatorerastelli/spotify-and-youtube                          Spotify and Youtube                                   9MB  2023-03-20 15:43:25           9136        334  1.0              
arnabchaki/data-science-salaries-2023                          Data Science Salaries 2023  💸                        25KB  2023-04-13 09:55:16           2507         69  1.0              
erdemtaha/cancer-data                                          Cancer Data                                          49KB  2023-03-22 07:57:00           4237         97  1.0              
evangower/premier-league-2022-2023                             Pr

TABLA IDENTITY: <BR>
Las variables en esta tabla son información de identidad: 
<BR><BR>
información de conexión de red (IP, ISP, Proxy, etc.) y firma digital (UA/
navegador/OS/versión, etc.) asociada con las transacciones.<BR><BR>
Son recopilados por el sistema de protección contra fraudes de Vesta y los socios de seguridad digital.<BR><BR>
(Los nombres de los campos están enmascarados y no se proporcionará el diccionario por pares para la protección de la privacidad y el acuerdo del contrato)<BR><BR>
DeviceType<BR>
DeviceInfo<BR>
id_12 - id_38<BR>

<BR><BR>


TABLA TRANSACCIONES: <BR>
TransactionDT: timedelta de una fecha y hora de referencia determinada (no una marca de tiempo real)<BR>
TransactionAMT: monto del pago de la transacción en USD<BR>
ProductCD: código de producto, el producto para cada transacción<BR>
card1 - card6: información de la tarjeta de pago, como tipo de tarjeta, categoría de tarjeta, banco emisor, país, etc.<BR>
dirección: dirección<BR>
distancia: distancia<BR>
P_ y (R__) emaildomain: dominio de correo electrónico del comprador y del destinatario<BR>
C1-C14: conteo, como cuántas direcciones se encuentran asociadas con la tarjeta de pago, etc. El significado real está enmascarado.<BR>
D1-D15: timedelta, como días entre transacciones anteriores, etc.<BR>
M1-M9: coincidencia, como nombres en la tarjeta y dirección, etc.<BR>
Vxxx: características completas diseñadas por Vesta, que incluyen clasificación, conteo y otras relaciones de entidad.<BR><BR>

Características categóricas:<BR>
ProductCD<BR>
card1 - card6<BR>
addr1, addr2<BR>
P_emaildomain<BR>
R_emaildomain<BR>
M1 - M9<BR>

In [16]:
#Vamos a leer los datos que tiene el dataset para empezar con el análisis explorartorio
df = pd.read_csv('sample_submission.csv')
df.head(10)

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5
5,3663554,0.5
6,3663555,0.5
7,3663556,0.5
8,3663557,0.5
9,3663558,0.5


In [17]:
df.columns#Vamos a indentificar las columnas de interes para hacer el análisis, en este caso como tenemos tantas columnas,
#vamos a basar el análisis en enfermedad de hipotiroidismo e hipertiroidismo (query_hypothyroid) (query_hyperthyroid)

Index(['TransactionID', 'isFraud'], dtype='object')

In [18]:
df.shape

(506691, 2)

Como podemos observar en nuestro dataset, cumplimos con la condición de que existan al menos 5000 instancias y 30 columnas, ya que para nuestro proyecto tenemos 9172 instancias o filas y 31 columnas.

In [19]:
#En este paso vamos a poder observar toda la información relacionada al dataframe que hemos cargado
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  506691 non-null  int64  
 1   isFraud        506691 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 7.7 MB


In [20]:
check_for_any_nan= df.isna().any().any()#Vamos a encontrar cuantos valores NaN existen en nuestro dataframe
total_nan_values = df.isna().sum().sum()#Vamos a sumar el total de valores NaN presentes en el dataframe

print("Existen valores en NaN: "+str(check_for_any_nan))
print("Total de valores en NaN: "+str(total_nan_values))

Existen valores en NaN: False
Total de valores en NaN: 0


Al chequear si existen valores en NaN, podemos ver que si existen y que tenemos 14629 valores en NaN, por lo que si hacemos cuentas, quitan la columan del id del paciente, tendriamos 30 columnas x 9172 filas lo que corresponde a 275,160 datos. Al evaluar cuantos valores NaN tenemos, se puede ver que son 14,629, por lo que este valor total de datos corresponde al 5,316% de datos faltantes respecto al total. Por lo que cumplimos tambien con la otra condición.

In [ ]:
df.isnull().sum() #Columnas con valores en NaN

In [ ]:
df.isnull().mean() * 100 #Promedio en las columnas con valores en NaN